In [5]:
# https://www.youtube.com/watch?v=8owQBFAHw7E
# loading libraries 
!pip install spektral==0.6.1

     |████████████████████████████████| 95 kB 2.3 MB/s 
  Attempting uninstall: spektral
    Found existing installation: spektral 1.0.7
    Uninstalling spektral-1.0.7:
      Successfully uninstalled spektral-1.0.7


In [1]:
import numpy as np
import tensorflow as tf
import spektral # to easly preprocess graphs and similar things 

In [2]:
adj, features, labels, train_mask, val_mask, test_mask = spektral.datasets.citation.load_data(dataset_name = 'cora')

Loading cora dataset
Pre-processing node features


In [3]:
features = features.todense()
adj = adj.todense() + np.eye(adj.shape[0])
features = features.astype('float32')
adj = adj.astype('float32')

In [4]:
print(features.shape)
print(adj.shape)
print(labels.shape)

print(np.sum(train_mask))
print(np.sum(val_mask))
print(np.sum(test_mask))


(2708, 1433)
(2708, 2708)
(2708, 7)
140
500
1000


In [5]:
# function that rerutn the cross entropy over the nodes of the graph

def masked_softmax_cross_entropy(logits, labels, mask):
  loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels = labels)
  mask = tf.cast(mask, dtype=tf.float32)
  mask /= tf.reduce_mean(mask)
  loss *= mask
  return tf.reduce_mean(loss)


# computing some accuracy metric, we only want this over val and test nodes  

def masked_accuracy(logits, labels, mask): 
  correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(labels,1)) # this compare argmax of the logits with argmax of the labels 
  accuracy_all = tf.cast(correct_prediction, tf.float32)
  mask = tf.cast(mask, dtype=tf.float32) # we need to cast the mask properly
  mask /= tf.reduce_mean(mask)
  accuracy_all *= mask
  return tf.reduce_mean(accuracy_all)

In [6]:
# now is time to define a very simple but GENERAL Graph NN layer 
# it will have a featyres matrix, and adjency matrix, a transformation that will be applied to every node and an activation function

def gnn(fts, adj, transform, activation): 
  seq_fts = transform(fts) # we transform each of the nodes individually
  ret_fts = tf.matmul(adj, seq_fts) # we multiply by an appropiate adjency matrix to combine the neighbors 
  return activation(ret_fts)

In [11]:
# defining a simple two layer gnn to clasify the cora dataset
# we can choose how many units our NN will compute foe each node, so how many dimensions of features  
# and a learning rate 

def train_cora(fts, adj, gnn_fn, units, epochs, lr):

  lyr_1 = tf.keras.layers.Dense(units)
  lyr_2 = tf.keras.layers.Dense(7) # compute the classification of each node, we need 7 outputs for 7 classes

  # now we can define the gnn that is used to solve this problem: 
  def cora_gnn(fts, adj): 
    hidden = gnn_fn(fts, adj, lyr_1, tf.nn.relu)
    logits = gnn_fn(hidden, adj, lyr_2, tf.identity) # it will project each node to 7 outputs 
    return logits 

  optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

  best_accuracy = 0.0

  for ep in range(epochs + 1): 
    with tf.GradientTape() as t:
      logits = cora_gnn(fts, adj)
      loss = masked_softmax_cross_entropy(logits, labels, train_mask)

    variables = t.watched_variables()
    grads = t.gradient(loss, variables)
    optimizer.apply_gradients(zip(grads,variables))

    logits = cora_gnn(fts, adj)
    val_accuracy = masked_accuracy(logits, labels, val_mask)
    test_accuracy = masked_accuracy(logits, labels, test_mask)

    if val_accuracy > best_accuracy: 
     best_accuracy = val_accuracy
     print('Epoch', ep, '| TRaining loss:', loss.numpy(), 
           '| Val accuracy:', val_accuracy.numpy(),'| Test accuracy:', test_accuracy.numpy())

In [12]:
train_cora(features, adj, gnn, 32, 200, 0.01)

Epoch 0 | TRaining loss: 1.9815553 | Val accuracy: 0.072 | Test accuracy: 0.09199999
Epoch 1 | TRaining loss: 2.0027316 | Val accuracy: 0.21599999 | Test accuracy: 0.22
Epoch 2 | TRaining loss: 1.746207 | Val accuracy: 0.484 | Test accuracy: 0.47300002
Epoch 3 | TRaining loss: 1.5606091 | Val accuracy: 0.58599997 | Test accuracy: 0.57600003
Epoch 7 | TRaining loss: 1.1307148 | Val accuracy: 0.6 | Test accuracy: 0.59900004
Epoch 8 | TRaining loss: 1.061319 | Val accuracy: 0.624 | Test accuracy: 0.633
Epoch 9 | TRaining loss: 0.98419034 | Val accuracy: 0.67599994 | Test accuracy: 0.677
Epoch 10 | TRaining loss: 0.9090838 | Val accuracy: 0.69 | Test accuracy: 0.70600003
Epoch 11 | TRaining loss: 0.84551406 | Val accuracy: 0.71 | Test accuracy: 0.719
Epoch 12 | TRaining loss: 0.77292615 | Val accuracy: 0.716 | Test accuracy: 0.73599994
Epoch 13 | TRaining loss: 0.71065366 | Val accuracy: 0.73800004 | Test accuracy: 0.747
Epoch 14 | TRaining loss: 0.64909166 | Val accuracy: 0.74999994 | Tes

In [13]:
# we change the adj matrix, this is worse because we dont explote the graph information 
train_cora(features, tf.eye(adj.shape[0]), gnn, 32, 200, 0.01)

Epoch 0 | TRaining loss: 1.9458084 | Val accuracy: 0.188 | Test accuracy: 0.18699999
Epoch 1 | TRaining loss: 1.9314784 | Val accuracy: 0.30199996 | Test accuracy: 0.28899997
Epoch 2 | TRaining loss: 1.9097278 | Val accuracy: 0.37199998 | Test accuracy: 0.36900002
Epoch 3 | TRaining loss: 1.8827087 | Val accuracy: 0.41799998 | Test accuracy: 0.38099998
Epoch 6 | TRaining loss: 1.78357 | Val accuracy: 0.44 | Test accuracy: 0.415
Epoch 7 | TRaining loss: 1.7443411 | Val accuracy: 0.45 | Test accuracy: 0.429
Epoch 8 | TRaining loss: 1.7020384 | Val accuracy: 0.45399997 | Test accuracy: 0.439
Epoch 9 | TRaining loss: 1.6569511 | Val accuracy: 0.45799997 | Test accuracy: 0.45999998
Epoch 10 | TRaining loss: 1.6093223 | Val accuracy: 0.47599998 | Test accuracy: 0.47100002
Epoch 11 | TRaining loss: 1.5593297 | Val accuracy: 0.482 | Test accuracy: 0.468
Epoch 12 | TRaining loss: 1.5070736 | Val accuracy: 0.48599997 | Test accuracy: 0.47300002
Epoch 13 | TRaining loss: 1.4526657 | Val accuracy:

In [14]:
# now we add the degree matrix, in order to demonstrate that is a good idea to normalize tha adjency matrix
deg = tf.reduce_sum(adj, axis=-1)
train_cora(features, adj / deg, gnn, 32, 200, 0.01)

Epoch 0 | TRaining loss: 1.9445227 | Val accuracy: 0.19599998 | Test accuracy: 0.21499999
Epoch 3 | TRaining loss: 1.8905356 | Val accuracy: 0.20799999 | Test accuracy: 0.24599999
Epoch 4 | TRaining loss: 1.8678628 | Val accuracy: 0.21599999 | Test accuracy: 0.25100002
Epoch 6 | TRaining loss: 1.8191857 | Val accuracy: 0.22199999 | Test accuracy: 0.265
Epoch 7 | TRaining loss: 1.7926778 | Val accuracy: 0.24 | Test accuracy: 0.28
Epoch 8 | TRaining loss: 1.7642461 | Val accuracy: 0.27 | Test accuracy: 0.304
Epoch 9 | TRaining loss: 1.7335058 | Val accuracy: 0.31 | Test accuracy: 0.34
Epoch 10 | TRaining loss: 1.7006091 | Val accuracy: 0.346 | Test accuracy: 0.36400002
Epoch 11 | TRaining loss: 1.6664574 | Val accuracy: 0.38799998 | Test accuracy: 0.39699998
Epoch 12 | TRaining loss: 1.6310173 | Val accuracy: 0.408 | Test accuracy: 0.42399997
Epoch 13 | TRaining loss: 1.5939981 | Val accuracy: 0.45599997 | Test accuracy: 0.45799997
Epoch 14 | TRaining loss: 1.5553218 | Val accuracy: 0.48

In [15]:
# finally we are going to use another normalization, that multiply both sides of adj matrix

norm_deg = tf.linalg.diag( 1.0 / tf.sqrt(deg))
norm_adj = tf.matmul(norm_deg, tf.matmul(adj,norm_deg))

train_cora(features, norm_adj, gnn, 32, 200, 0.01)

Epoch 0 | TRaining loss: 1.945597 | Val accuracy: 0.22799999 | Test accuracy: 0.247
Epoch 1 | TRaining loss: 1.9315991 | Val accuracy: 0.248 | Test accuracy: 0.27499998
Epoch 2 | TRaining loss: 1.9154929 | Val accuracy: 0.296 | Test accuracy: 0.314
Epoch 3 | TRaining loss: 1.8963563 | Val accuracy: 0.328 | Test accuracy: 0.335
Epoch 4 | TRaining loss: 1.8763411 | Val accuracy: 0.33799997 | Test accuracy: 0.347
Epoch 5 | TRaining loss: 1.8544222 | Val accuracy: 0.362 | Test accuracy: 0.361
Epoch 6 | TRaining loss: 1.8303219 | Val accuracy: 0.37199998 | Test accuracy: 0.38300002
Epoch 7 | TRaining loss: 1.805059 | Val accuracy: 0.39399996 | Test accuracy: 0.39799997
Epoch 8 | TRaining loss: 1.7780575 | Val accuracy: 0.41799995 | Test accuracy: 0.441
Epoch 9 | TRaining loss: 1.7491691 | Val accuracy: 0.45599997 | Test accuracy: 0.49899998
Epoch 10 | TRaining loss: 1.7188176 | Val accuracy: 0.52599996 | Test accuracy: 0.542
Epoch 11 | TRaining loss: 1.686837 | Val accuracy: 0.576 | Test ac